In [ ]:
!pip install transformers==4.3.2
import torch
import io
import torch.nn.functional as F
import random
import numpy as np
import time
import math
import datetime
import torch.nn as nn
from transformers import *
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix


import numpy as np
import pandas as pd
import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report,accuracy_score,precision_score,recall_score,f1_score
from collections import defaultdict


#!pip install torch==1.7.1+cu101 torchvision==0.8.2+cu101 -f https://download.pytorch.org/whl/torch_stable.html
#!pip install sentencepiece
import pandas as pd
##Set random values
seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
if torch.cuda.is_available():
  torch.cuda.manual_seed_all(seed_val)

In [ ]:
# If there's a GPU available...
if torch.cuda.is_available():    
    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla V100-SXM2-16GB


In [ ]:

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#--------------------------------
#  Transformer parameters
#--------------------------------
max_seq_length = 200
batch_size = 32
NUM_CLS = 3
#--------------------------------
#  GAN-BERT specific parameters
#--------------------------------
# number of hidden layers in the generator, 
# each of the size of the output space
num_hidden_layers_g = 1; 
# number of hidden layers in the discriminator, 
# each of the size of the input space
num_hidden_layers_d = 1; 
# size of the generator's input noisy vectors
noise_size = 100
# dropout to be applied to discriminator's input vectors
out_dropout_rate = 0.4

# Replicate labeled data to balance poorly represented datasets, 
# e.g., less than 1% of labeled material
apply_balance = True

#--------------------------------
#  Optimization parameters
#--------------------------------


learning_rate_discriminator = 5e-5
learning_rate_generator = 3e-5
epsilon = 1e-8
num_train_epochs = 20
multi_gpu = True
# Scheduler
apply_scheduler = True
warmup_proportion = 0.1
# Print
print_each_n_step = 10

#--------------------------------
#  Adopted Tranformer model
#--------------------------------

model_name = "bert-base-cased"
transformer = AutoModel.from_pretrained(model_name)

from sklearn.datasets import fetch_20newsgroups





In [ ]:




class Data_Loader():




    def __init__(self,NUM_LABEL = 20,NUM_UNLABEL = 5000,NUM_CLS = 3 ):

      self.NUM_LABEL = NUM_LABEL
      self.NUM_UNLABEL = NUM_UNLABEL
      self.NUM_CLS = NUM_CLS
      self.tokenizer = AutoTokenizer.from_pretrained(model_name)




    def yahoo_data_reader(self):


      df =  pd.read_excel("/content/drive/MyDrive/IMDB/train.xlsx")
      df_test = pd.read_excel("/content/drive/MyDrive/IMDB/test.xlsx")

      df.Sentiment.replace(to_replace=['neg', 'pos'], value=[1, 2], inplace=True)
      df_test.Sentiment.replace(to_replace=['neg', 'pos'], value=[1, 2], inplace=True)


      df= df.rename(columns={'Sentiment': 'Target','Reviews':'comment'}).iloc[1: , :].reset_index(drop = True)
      df_test= df_test.rename(columns={'Sentiment': 'Target','Reviews':'comment'}).iloc[1: , :].reset_index(drop = True)




      print(df)
      print(df_test)

      return df,df_test




    def sample_data( self,data_frame,number = 20,number_of_unlabel = 5000 ,number_of_classes = 2):

      df_label = pd.DataFrame({'Target' : [],
                              'comment':[]})

      df_unlabel = pd.DataFrame({'Target' : [],
                              'comment':[]})

      num_unlabel_per_class = int(number_of_unlabel/number_of_classes)

      for i in range(1,number_of_classes):

        df_subset = data_frame[data_frame["Target"] ==i ].sample(n=number,random_state= 42,replace=False)
        df_label = df_label.append(df_subset,ignore_index=True)

        df_subset1 = data_frame[data_frame["Target"] ==i ].sample(n=num_unlabel_per_class,random_state= 21,replace=False)
        df_unlabel = df_unlabel.append(df_subset1,ignore_index=True)


      return df_label,df_unlabel



    def example_maker(self,dataframe):

      ll =[]
      for i in range(len(dataframe)):
        ll.append((dataframe.comment[i],dataframe.Target[i]))
          

      return ll  

    def example_maker1(self,dataframe):

      ll =[]
      for i in range(len(dataframe)):
        ll.append((dataframe.comment[i],0))
          

      return ll    




    def generate_data_loader(self,input_examples, label_masks, do_shuffle = False, balance_label_examples = False):
      '''
      Generate a Dataloader given the input examples, eventually masked if they are 
      to be considered NOT labeled.
      '''
      examples = []

      # Count the percentage of labeled examples  
      num_labeled_examples = 0
      for label_mask in label_masks:
        if label_mask: 
          num_labeled_examples += 1
      label_mask_rate = num_labeled_examples/len(input_examples)

      # if required it applies the balance
      for index, ex in enumerate(input_examples): 
        # print("index",index)
        # print("ex",ex)

        if label_mask_rate == 1 or not balance_label_examples:
          examples.append((ex, label_masks[index]))
        else:
          # IT SIMULATE A LABELED EXAMPLE
          if label_masks[index]:
            balance = int(1/label_mask_rate)
            balance = int(math.log(balance,2))
            if balance < 1:
              balance = 1
            for b in range(0, int(balance)):
              examples.append((ex, label_masks[index]))
          else:
            examples.append((ex, label_masks[index]))

      #-----------------------------------------------
      # Generate input examples to the Transformer
      #-----------------------------------------------
      input_ids = []
      input_mask_array = []
      label_mask_array = []
      label_id_array = []

      # Tokenization
      count = 0 
      for (text, label_mask) in examples:
        
        count = count+1
        if(type(text[0]) != int):
          encoded_sent = self.tokenizer.encode(text[0], add_special_tokens=True, max_length=max_seq_length, padding="max_length", truncation=True)
          if len(encoded_sent) != 200:
            print(len(encoded_sent))
          input_ids.append(encoded_sent)
          label_id_array.append(text[1])
          label_mask_array.append(label_mask)
        else:
          print(text[0])

      # Attention to token (to ignore padded input wordpieces)
      for sent in input_ids:
        att_mask = [int(token_id > 0) for token_id in sent]                          
        input_mask_array.append(att_mask)
      # Convertion to Tensor

      input_ids = torch.tensor(input_ids) 
      input_mask_array = torch.tensor(input_mask_array)
      label_id_array = torch.tensor(label_id_array, dtype=torch.long)
      label_mask_array = torch.tensor(label_mask_array)

      # Building the TensorDataset
      dataset = TensorDataset(input_ids, input_mask_array, label_id_array, label_mask_array)

      if do_shuffle:
        sampler = RandomSampler
      else:
        sampler = SequentialSampler

      # Building the DataLoader
      return DataLoader(
                  dataset,  # The training samples.
                  batch_size = batch_size,
                  shuffle=True) # Trains with this batch size.

    def data_loader(self):

      df,test_examples1  = self.yahoo_data_reader()
      labeled_examples1,unlabeled_examples1 = self.sample_data(df,number = NUM_LABEL ,number_of_unlabel = 5000,number_of_classes =NUM_CLS )

      ########################
      labeled_examples = self.example_maker(labeled_examples1)
      unlabeled_examples = self.example_maker1(unlabeled_examples1)
      test_examples = self.example_maker(test_examples1)
      ##########################
      #------------------------------
      #   Load the test dataset
      #------------------------------

      train_examples = labeled_examples


      train_label_masks = np.ones(len(labeled_examples), dtype=bool)

      if unlabeled_examples:
        train_examples = train_examples + unlabeled_examples
        #The unlabeled (train) dataset is assigned with a mask set to False
        tmp_masks = np.zeros(len(unlabeled_examples), dtype=bool)
        train_label_masks = np.concatenate([train_label_masks,tmp_masks])

      train_dataloader = self.generate_data_loader(train_examples, train_label_masks, do_shuffle = True, balance_label_examples = apply_balance)

      #The labeled (test) dataset is assigned with a mask set to True
      test_label_masks = np.ones(len(test_examples), dtype=bool)

      test_dataloader = self.generate_data_loader(test_examples, test_label_masks, do_shuffle = False, balance_label_examples = False)
      ####################################################

      return train_dataloader,test_dataloader,train_examples

In [ ]:
def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))


In [ ]:


#------------------------------
#   The Generator as in 
#   https://www.aclweb.org/anthology/2020.acl-main.191/
#   https://github.com/crux82/ganbert
#------------------------------


class Generator(nn.Module):
    def __init__(self, noise_size=100, output_size=512, hidden_sizes=[512], dropout_rate=0.1):
        super(Generator, self).__init__()
        layers = []
        hidden_sizes = [noise_size] + hidden_sizes
        for i in range(len(hidden_sizes)-1):
            layers.extend([nn.Linear(hidden_sizes[i], hidden_sizes[i+1]), nn.LeakyReLU(0.2, inplace=True), nn.Dropout(dropout_rate)])
        #hidden_sizes[-1] = 512
        layers.append(nn.Linear(hidden_sizes[-1],output_size))
        self.layers = nn.Sequential(*layers)
        print("G:layers",layers)
        print("G:self.layers", self.layers)

    def forward(self, noise):
        output_rep = self.layers(noise)
        return output_rep
#------------------------------
#   The Discriminator
#   https://www.aclweb.org/anthology/2020.acl-main.191/
#   https://github.com/crux82/ganbert
#------------------------------
class Discriminator(nn.Module):
    def __init__(self, input_size=512, hidden_sizes=[512], num_labels=2, dropout_rate=0.1):
        super(Discriminator, self).__init__()
        self.input_dropout = nn.Dropout(p=dropout_rate)
        layers = []
        hidden_sizes = [input_size] + hidden_sizes
        for i in range(len(hidden_sizes)-1):
            layers.extend([nn.Linear(hidden_sizes[i], hidden_sizes[i+1]), nn.LeakyReLU(0.2, inplace=True), nn.Dropout(dropout_rate)])

        self.layers = nn.Sequential(*layers) #per il flatten
        self.logit = nn.Linear(hidden_sizes[-1],num_labels+1) # +1 for the probability of this sample being fake/real.
        self.softmax = nn.Softmax(dim=-1)

        self.attr = nn.Linear(768 + 3,768 )

        self.relu= nn.LeakyReLU(0.2, inplace=True)

        # print("D:layers",layers)
        # print("D:self.layers", self.layers)
        # print("self.logit",self.logit)


    def forward(self, input_rep, label = None,label_embed = None):


        input_rep = self.input_dropout(input_rep)
        last_rep = self.layers(input_rep)
        logits = self.logit(last_rep)
        probs = self.softmax(logits)


        return last_rep, logits, probs


    

      








def noise_gen(input_shape, noise_size, device):

  return torch.zeros(input_shape,noise_size, device=device).uniform_(0, 1)








def train(transformer,generator,discriminator,train_iter,gen_optimizer,dis_optimizer,scheduler_d,scheduler_g):

  transformer.train() 
  generator.train()
  discriminator.train()

  t0 = time.time()

  # Reset the total loss for this epoch.
  tr_g_loss = 0
  tr_d_loss = 0

  for step, batch in enumerate(train_iter):


      b_input_ids = batch[0].to(device)
      b_input_mask = batch[1].to(device)
      b_labels = batch[2].to(device)
      b_label_mask = batch[3].to(device)

      # label_embed = generator.label_embedding


      model_outputs = transformer(b_input_ids, attention_mask=b_input_mask)
      hidden_states = model_outputs[-1]

      #NDA = hidden_states.detach()
      NDA = hidden_states
      #add label stuff
      real_feature, real_logits, real_probs = discriminator(hidden_states)


      #---------------------------------
      # Simple Generator
      #---------------------------------

      
      #fake data the same batch size of unlabel data
      noise = noise_gen(b_input_ids.shape[0], noise_size, device)
      
      # Gnerate Fake data
      gen_rep = generator(noise)

      alpha = 0.9 
      l = np.random.beta(alpha, alpha)
      l = max(l, 1-l)
      #l= 0.9
      l= 0.85
      neg_aug = l * gen_rep + (1 - l) * NDA
      neg_aug = neg_aug.to(device)
      fake_feature, fake_logits, fake_probs  = discriminator(neg_aug)



      #---------------------------------
      # Generator's LOSS estimation
      #---------------------------------
      g_loss_d =  -1 * torch.mean(torch.log(1 - fake_probs[:, -1] + epsilon))
      
      g_feat_reg = torch.mean(torch.pow(torch.mean(real_feature, dim=0) - torch.mean(fake_feature, dim=0), 2))
      g_loss = g_loss_d + g_feat_reg 




      #---------------------------------
      # Discriminator's LOSS estimation
      #---------------------------------
      D_L_Supervised = supervised_loss(real_logits,b_labels,b_label_mask)
      D_L_unsupervised1U = -1 * torch.mean(torch.log(1 - real_probs[:, -1] + epsilon))
      D_L_unsupervised2U = -1 * torch.mean(torch.log(fake_probs[:, -1] + epsilon))
      # D_L_unsupervised2U_simple = -1 * torch.mean(torch.log(D_fake_probs1[:, -1] + epsilon))
      d_loss = D_L_Supervised + D_L_unsupervised1U + D_L_unsupervised2U


      #---------------------------------
      #  OPTIMIZATION
      #---------------------------------
      # Avoid gradient accumulation
      gen_optimizer.zero_grad()
      dis_optimizer.zero_grad()

      # Calculate weigth updates
      # retain_graph=True is required since the underlying graph will be deleted after backward
      g_loss.backward(retain_graph=True)
      # d_loss.backward() 
      d_loss.backward()
      
      # Apply modifications
      gen_optimizer.step()
      dis_optimizer.step()

      # Save the losses to print them later
      tr_g_loss += g_loss.item()
      tr_d_loss += d_loss.item()


      # Update the learning rate with the scheduler
      if apply_scheduler:
        scheduler_d.step()
        scheduler_g.step()

  # Calculate the average loss over all of the batches.
  ###### it is very important how many times we use label data and how we calculate the loss
  avg_train_loss_g = tr_g_loss / len(train_iter)
  avg_train_loss_d = tr_d_loss/ len(train_iter)             

  # Measure how long this epoch took.
  training_time = format_time(time.time() - t0)

  print("")
  print("  Average training loss generetor: {0:.3f}".format(avg_train_loss_g))
  print("  Average training loss discriminator: {0:.3f}".format(avg_train_loss_d))

  print("  Training epcoh took: {:}".format(training_time))


  result_dic = {
      
      'Training Loss generator': avg_train_loss_g,
      'Training Loss discriminator sup': avg_train_loss_d,
      'Training Time': training_time,
  }

  return result_dic
    











def supervised_loss(D_real_logits,b_labels,b_label_mask):

  # Disciminator's LOSS estimation
  logits = D_real_logits[:,0:-1]
  # print("logits",logits.shape)
  log_probs = F.log_softmax(logits, dim=-1)

  label2one_hot = torch.nn.functional.one_hot(b_labels, NUM_CLS)
  
  per_example_loss = -torch.sum(label2one_hot * log_probs, dim=-1)

  per_example_loss = torch.masked_select(per_example_loss, b_label_mask.to(device))
  
  labeled_example_count = per_example_loss.type(torch.float32).numel()
  
  if labeled_example_count == 0:
    D_L_Supervised = 0
  else:
    D_L_Supervised = torch.div(torch.sum(per_example_loss.to(device)), labeled_example_count)

  return D_L_Supervised


#loss
nll_loss = torch.nn.CrossEntropyLoss(ignore_index=-1)

def valid(transformer,generator,discriminator,test_dataloader,nll_loss):

  print("Running Test...")

  t0 = time.time()

 
  transformer.eval() #maybe redundant
  discriminator.eval()
  generator.eval()


  # Tracking variables 
  total_test_accuracy = 0
  total_test_loss = 0
  nb_test_steps = 0

  all_preds = []
  all_labels_ids = []
  accurcies = []

  #loss
  nll_loss = torch.nn.CrossEntropyLoss(ignore_index=-1)

  # Evaluate data for one epoch
  for batch in test_dataloader:
      
      # Unpack this training batch from our dataloader. 
      b_input_ids = batch[0].to(device)
      b_input_mask = batch[1].to(device)
      b_labels = batch[2].to(device)
      
      # Tell pytorch not to bother with constructing the compute graph during
      # the forward pass, since this is only needed for backprop (training).
      with torch.no_grad():        
          model_outputs = transformer(b_input_ids, attention_mask=b_input_mask)
          hidden_states = model_outputs[-1]
          _, logits, probs = discriminator(hidden_states)
          filtered_logits = logits[:,0:-1]
          #print("filtered_logits",filtered_logits)
          total_test_loss += nll_loss(filtered_logits, b_labels)
          
      # Accumulate the predictions and the input labels
      _, preds = torch.max(filtered_logits, 1)
      # print("preds",preds)
      # print("b_labels",b_labels)

      all_preds += preds.detach().cpu()
      all_labels_ids += b_labels.detach().cpu()

  # Report the final accuracy for this validation run.
  all_preds = torch.stack(all_preds).numpy()
  all_labels_ids = torch.stack(all_labels_ids).numpy()
  test_accuracy = np.sum(all_preds == all_labels_ids) / len(all_preds)
  accurcies.append(test_accuracy)
  print("  Accuracy: {0:.3f}".format(test_accuracy))

  F1 = f1_score(all_labels_ids, all_preds,average="weighted")
  print("F1: ",F1)

  # Calculate the average loss over all of the batches.
  avg_test_loss = total_test_loss / len(test_dataloader)
  avg_test_loss = avg_test_loss.item()
  
  # Measure how long the validation run took.
  test_time = format_time(time.time() - t0)
  
  print("  Test Loss: {0:.3f}".format(avg_test_loss))
  print("  Test took: {:}".format(test_time))

  return test_accuracy










In [ ]:
# #number_of_labeled_data for each class
# NUM_LABEL = 20

# #number_of_unlabeled_data 
# NUM_UNLABEL = 5000

# #number of classes +1 
# NUM_CLS = 3
# train_dataloader,test_dataloader,train_examples = Data_Loader(NUM_LABEL,NUM_UNLABEL ,NUM_CLS).data_loader()



# # The config file is required to get the dimension of the vector produced by 
# # the underlying transformer
# config = AutoConfig.from_pretrained(model_name)
# hidden_size = int(config.hidden_size)

# # Define the number and width of hidden layers
# hidden_levels_g = [hidden_size for i in range(0, num_hidden_layers_g)]
# hidden_levels_d = [hidden_size for i in range(0, num_hidden_layers_d)]

# # print("hidden_levels_g",hidden_levels_g)
# # print("hidden_levels_d",hidden_levels_d)

# #-------------------------------------------------
# #   Instantiate the Generator and Discriminator
# #-------------------------------------------------

# generator = Generator(noise_size=noise_size, output_size=hidden_size, hidden_sizes=[768], dropout_rate=out_dropout_rate)

# discriminator = Discriminator(input_size=hidden_size, hidden_sizes=[768], num_labels=NUM_CLS , dropout_rate=out_dropout_rate)

# # Put everything in the GPU if available
# if torch.cuda.is_available():    
#   generator.cuda()
#   discriminator.cuda()
#   transformer.cuda()

#   if multi_gpu:
#     transformer = torch.nn.DataParallel(transformer)


# # Measure the total training time for the whole run.
# total_t0 = time.time()

# #models parameters
# transformer_vars = [i for i in transformer.parameters()]
# d_vars = transformer_vars + [v for v in discriminator.parameters()]
# g_vars = [v for v in generator.parameters()]

# #optimizer
# dis_optimizer = torch.optim.AdamW(d_vars, lr=learning_rate_discriminator)
# gen_optimizer = torch.optim.AdamW(g_vars, lr=learning_rate_generator) 


# #scheduler
# if apply_scheduler:
#   num_train_examples = len(train_examples)
#   num_train_steps = int(num_train_examples / batch_size * num_train_epochs)
#   num_warmup_steps = int(num_train_steps * warmup_proportion)

#   scheduler_d = get_constant_schedule_with_warmup(dis_optimizer, 
#                                            num_warmup_steps = num_warmup_steps)
#   scheduler_g = get_constant_schedule_with_warmup(gen_optimizer, 
#                                            num_warmup_steps = num_warmup_steps)


# # PATH ="/content/drive/MyDrive/20news/NDABERT"

# PATH ="/content/drive/MyDrive/ex2/"

In [ ]:
# max_accuracy = 0
# for epoch_i in range(0, 15):

#   train(transformer,generator,discriminator,train_dataloader,gen_optimizer,dis_optimizer,scheduler_d,scheduler_g)
#   test_accuracy = valid(transformer,generator,discriminator,test_dataloader,nll_loss)
#   if test_accuracy >= max_accuracy:
#     max_accuracy = test_accuracy
#     torch.save(generator, PATH +"NDA_imdb_generator_20.pt")
#     torch.save(discriminator, PATH +"NDA_imdb_discriminator_20.pt")
#     torch.save(transformer, PATH +"NDA_imdb_transformer_20.pt")








In [ ]:
#number_of_labeled_data for each class
NUM_LABEL = 1000

#number_of_unlabeled_data 
NUM_UNLABEL = 5000

#number of classes +1 
NUM_CLS = 3
train_dataloader,test_dataloader,train_examples = Data_Loader(NUM_LABEL,NUM_UNLABEL ,NUM_CLS).data_loader()



# The config file is required to get the dimension of the vector produced by 
# the underlying transformer
config = AutoConfig.from_pretrained(model_name)
hidden_size = int(config.hidden_size)

# Define the number and width of hidden layers
hidden_levels_g = [hidden_size for i in range(0, num_hidden_layers_g)]
hidden_levels_d = [hidden_size for i in range(0, num_hidden_layers_d)]

# print("hidden_levels_g",hidden_levels_g)
# print("hidden_levels_d",hidden_levels_d)

#-------------------------------------------------
#   Instantiate the Generator and Discriminator
#-------------------------------------------------

generator = Generator(noise_size=noise_size, output_size=hidden_size, hidden_sizes=[768], dropout_rate=out_dropout_rate)

discriminator = Discriminator(input_size=hidden_size, hidden_sizes=[768], num_labels=NUM_CLS , dropout_rate=out_dropout_rate)

# Put everything in the GPU if available
if torch.cuda.is_available():    
  generator.cuda()
  discriminator.cuda()
  transformer.cuda()

  if multi_gpu:
    transformer = torch.nn.DataParallel(transformer)


# Measure the total training time for the whole run.
total_t0 = time.time()

#models parameters
transformer_vars = [i for i in transformer.parameters()]
d_vars = transformer_vars + [v for v in discriminator.parameters()]
g_vars = [v for v in generator.parameters()]

#optimizer
dis_optimizer = torch.optim.AdamW(d_vars, lr=learning_rate_discriminator)
gen_optimizer = torch.optim.AdamW(g_vars, lr=learning_rate_generator) 


#scheduler
if apply_scheduler:
  num_train_examples = len(train_examples)
  num_train_steps = int(num_train_examples / batch_size * num_train_epochs)
  num_warmup_steps = int(num_train_steps * warmup_proportion)

  scheduler_d = get_constant_schedule_with_warmup(dis_optimizer, 
                                           num_warmup_steps = num_warmup_steps)
  scheduler_g = get_constant_schedule_with_warmup(gen_optimizer, 
                                           num_warmup_steps = num_warmup_steps)


# PATH ="/content/drive/MyDrive/20news/NDABERT"

PATH ="/content/drive/MyDrive/ex2/"

                                                 comment  Target
0      Mere thoughts of "Going Overboard" (aka "Babes...       1
1      Why does this movie fall WELL below standards?...       1
2      Wow and I thought that any Steven Segal movie ...       1
3      The story is seen before, but that does'n matt...       1
4      Like so many media experiments, this amateuris...       1
...                                                  ...     ...
24994  Everyone plays their part pretty well in this ...       2
24995  It happened with Assault on Prescient 13 in 20...       1
24996  My God. This movie was awful. I can't complain...       1
24997  When I first popped in Happy Birthday to Me, I...       1
24998  So why does this show suck? Unfortunately, tha...       1

[24999 rows x 2 columns]
                                                 comment  Target
0      After realizing what is going on around us ......       2
1      I grew up watching the original Disney Cindere...       1

In [ ]:
max_accuracy = 0
for epoch_i in range(0, 15):

  train(transformer,generator,discriminator,train_dataloader,gen_optimizer,dis_optimizer,scheduler_d,scheduler_g)
  test_accuracy = valid(transformer,generator,discriminator,test_dataloader,nll_loss)
  if test_accuracy >= max_accuracy:
    max_accuracy = test_accuracy
    torch.save(generator, PATH +"NDA_imdb_generator_1000.pt")
    torch.save(discriminator, PATH +"NDA_imdb_discriminator_1000.pt")
    torch.save(transformer, PATH +"NDA_imdb_transformer_1000.pt")









  Average training loss generetor: 0.464
  Average training loss discriminator: 2.092
  Training epcoh took: 0:01:31
Running Test...
  Accuracy: 0.824
F1:  0.821636075273582
  Test Loss: 0.417
  Test took: 0:01:27

  Average training loss generetor: 0.738
  Average training loss discriminator: 1.120
  Training epcoh took: 0:01:32
Running Test...
  Accuracy: 0.853
F1:  0.8523697847987189
  Test Loss: 0.391
  Test took: 0:01:27

  Average training loss generetor: 0.722
  Average training loss discriminator: 0.983
  Training epcoh took: 0:01:32
Running Test...
  Accuracy: 0.863
F1:  0.8626892408079944
  Test Loss: 0.372
  Test took: 0:01:27

  Average training loss generetor: 0.714
  Average training loss discriminator: 0.843
  Training epcoh took: 0:01:32
Running Test...
  Accuracy: 0.846
F1:  0.844877544347417
  Test Loss: 0.401
  Test took: 0:01:27

  Average training loss generetor: 0.712
  Average training loss discriminator: 0.827
  Training epcoh took: 0:01:32
Running Test...
  Ac